# Prediction Model

We will be creating multiple regression models that will try and predict housing prices based on the features in our cleaned datasets. The models will be evaluated based on the root mean squared error (RMSE) of their predictions against the validation set. Once the best production model is found, the final model will then be retrained using the entire training dataset then its test set predictions will be submitted to Kaggle to determine the actual test score.

## Contents:
- [Regression Modelling](#Regression-Modelling)
- [Initial Kaggle Submission](#Initial-Kaggle-Submission)
- [Model Improvement](#Model-Improvement)
- [Final Kaggle Submission](#Final-Kaggle-Submission)
- [Conclusions](#Conclusions)

------------------------------------------------

In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.feature_selection import RFE, RFECV
import sklearn.metrics as metrics
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

In [3]:
train=pd.read_csv('../datasets/training_model.csv')
valid=pd.read_csv('../datasets/validation_model.csv')
test=pd.read_csv('../datasets/test_kaggle.csv')
ori_test=pd.read_csv('../datasets/test.csv')

In [4]:
X_train = train.drop(columns='saleprice')
X_valid = valid.drop(columns='saleprice')
y_train = train['saleprice']
y_valid = valid['saleprice']

X_test = test

In [5]:
#checking the number of rows for each dataset
print(f'Number of columns for X_train is {X_train.shape[1]}')
print(f'Number of columns for X_valid is {X_valid.shape[1]}')
print(f'Number of columns for X_train is {X_test.shape[1]}')

Number of columns for X_train is 191
Number of columns for X_valid is 191
Number of columns for X_train is 191


------------------------------------------------
## Regression Modelling

### Linear Regression

In [6]:
# Instantiate
lr = LinearRegression()

#cross-validation
np.abs(cross_val_score(lr, X_train, y_train, scoring='neg_root_mean_squared_error').mean())

328349940403081.9

In [7]:
#check r2 scores
lr_scores = cross_val_score(lr, X_train, y_train, cv=3)

print(lr_scores)
lr_scores.mean() 

[-3.53790996e+20 -3.01238882e+19 -1.56359302e+17]


-1.2802374793495893e+20

In [8]:
# Evaluation against validation set
lr.fit(X_train, y_train)

np.sqrt(metrics.mean_squared_error(y_valid, lr.predict(X_valid)))

18200765718856.78

### Ridge Regression

In [9]:
# Finding best alpha term
r_alphas = np.linspace(.1, 10, 100)
ridge_cv = RidgeCV(alphas=r_alphas)
ridge_cv.fit(X_train, y_train)

RidgeCV(alphas=array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.1,
        1.2,  1.3,  1.4,  1.5,  1.6,  1.7,  1.8,  1.9,  2. ,  2.1,  2.2,
        2.3,  2.4,  2.5,  2.6,  2.7,  2.8,  2.9,  3. ,  3.1,  3.2,  3.3,
        3.4,  3.5,  3.6,  3.7,  3.8,  3.9,  4. ,  4.1,  4.2,  4.3,  4.4,
        4.5,  4.6,  4.7,  4.8,  4.9,  5. ,  5.1,  5.2,  5.3,  5.4,  5.5,
        5.6,  5.7,  5.8,  5.9,  6. ,  6.1,  6.2,  6.3,  6.4,  6.5,  6.6,
        6.7,  6.8,  6.9,  7. ,  7.1,  7.2,  7.3,  7.4,  7.5,  7.6,  7.7,
        7.8,  7.9,  8. ,  8.1,  8.2,  8.3,  8.4,  8.5,  8.6,  8.7,  8.8,
        8.9,  9. ,  9.1,  9.2,  9.3,  9.4,  9.5,  9.6,  9.7,  9.8,  9.9,
       10. ]))

In [10]:
# Best alpha
ridge_cv.alpha_

4.5

In [11]:
# Instantiate
ridge = Ridge(alpha=ridge_cv.alpha_)

#cross-validation
np.abs(cross_val_score(ridge, X_train, y_train, scoring='neg_root_mean_squared_error').mean())

24825.246819215317

In [12]:
#check r2 scores
ridge_scores = cross_val_score(ridge, X_train, y_train, cv=3)

print(ridge_scores)
ridge_scores.mean() 

[0.90716391 0.91174741 0.89679367]


0.9052350010580512

In [13]:
## Evaluation against validation set
ridge.fit(X_train, y_train)

np.sqrt(metrics.mean_squared_error(y_valid, ridge.predict(X_valid)))

81394.86679720023

### Lasso Regression

In [14]:
# Finding best alpha term
lasso_cv = LassoCV(n_alphas=200)

#fit model us
lasso_cv.fit(X_train, y_train)

LassoCV(n_alphas=200)

In [15]:
# Best alpha
lasso_cv.alpha_

65.46257920593182

In [16]:
# Instantiate
lasso = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)

#cross-validation
np.abs(cross_val_score(lasso, X_train, y_train, scoring='neg_root_mean_squared_error').mean())

24682.69122736121

In [17]:
#check r2 scores
lasso_scores = cross_val_score(lasso, X_train, y_train, cv=3)

print(lasso_scores)
lasso_scores.mean() 

[0.9060296  0.91431536 0.90206278]


0.9074692444926035

In [18]:
## Evaluation against validation set
lasso.fit(X_train, y_train)

np.sqrt(metrics.mean_squared_error(y_valid, lasso.predict(X_valid)))

78757.68236739191

### Elastic Net

In [19]:
# Finding best alpha term
enet_alphas = np.linspace(0.5, 1.0, 100)# Return evenly spaced numbers over a specified interval
enet_cv = ElasticNetCV(alphas=enet_alphas, cv=5) #l1_ratiofloat, default=0.5

# Fit model using optimal alpha.
enet_cv.fit(X_train, y_train)

ElasticNetCV(alphas=array([0.5       , 0.50505051, 0.51010101, 0.51515152, 0.52020202,
       0.52525253, 0.53030303, 0.53535354, 0.54040404, 0.54545455,
       0.55050505, 0.55555556, 0.56060606, 0.56565657, 0.57070707,
       0.57575758, 0.58080808, 0.58585859, 0.59090909, 0.5959596 ,
       0.6010101 , 0.60606061, 0.61111111, 0.61616162, 0.62121212,
       0.62626263, 0.63131313, 0.63636364, 0.64141414, 0.64646465,
       0....
       0.82828283, 0.83333333, 0.83838384, 0.84343434, 0.84848485,
       0.85353535, 0.85858586, 0.86363636, 0.86868687, 0.87373737,
       0.87878788, 0.88383838, 0.88888889, 0.89393939, 0.8989899 ,
       0.9040404 , 0.90909091, 0.91414141, 0.91919192, 0.92424242,
       0.92929293, 0.93434343, 0.93939394, 0.94444444, 0.94949495,
       0.95454545, 0.95959596, 0.96464646, 0.96969697, 0.97474747,
       0.97979798, 0.98484848, 0.98989899, 0.99494949, 1.        ]),
             cv=5)

In [20]:
# Best alpha
enet_cv.alpha_

0.5

In [21]:
# Instantiate
enet = ElasticNet(alpha=enet_cv.alpha_)

enet_scores = cross_val_score(lasso, X_train, y_train, cv=7)

In [22]:
#cross-validation
np.abs(cross_val_score(enet, X_train, y_train, scoring='neg_root_mean_squared_error').mean())

26986.228164549004

In [23]:
#check r2 score
enet_scores = cross_val_score(enet, X_train, y_train, cv=7)

print(enet_scores)
enet_scores.mean() 

[0.89884324 0.85208447 0.89749746 0.90632427 0.88709159 0.86739244
 0.91346104]


0.8889563576366666

In [24]:
## Evaluation against validation set
enet.fit(X_train, y_train)

np.sqrt(metrics.mean_squared_error(y_valid, enet.predict(X_valid)))

58293.51542089644


### Initial Kaggle Submission

In [27]:
def kaggle_submission(preds, model_name):
    
    submission = pd.DataFrame(data=preds)
    submission = pd.merge(ori_test['Id'], submission, left_index = True, right_index = True)
    
    submission.rename({'Id' : 'ID',
                      0 : 'SalePrice'},
                     inplace = True,
                     axis = 1)
    
    submission.to_csv(f'../datasets/submission_{model_name}.csv', index=False)
    
    return submission

In [28]:
test_preds_lr = lr.predict(X_test)
kaggle_submission(test_preds_lr, 'linear_reg_01')

,ID,SalePrice
0,2658,116977.0
1,2718,165235.0
2,2414,226768.0
3,1989,108813.0
4,625,176170.0
...,...,...
873,1662,174457.0
874,1234,214885.0
875,1373,123810.0
876,1672,119036.0


In [29]:
test_preds_ridge = ridge.predict(X_test)
kaggle_submission(test_preds_ridge, 'ridge_01')

,ID,SalePrice
0,2658,119823.519988
1,2718,162643.733657
2,2414,228753.268910
3,1989,110789.166687
4,625,175069.051031
...,...,...
873,1662,175625.971555
874,1234,219981.573215
875,1373,124785.062032
876,1672,119634.484012


In [30]:
test_preds_lasso = lasso.predict(X_test)
kaggle_submission(test_preds_lasso, 'lasso_01')

,ID,SalePrice
0,2658,124701.714780
1,2718,161796.819796
2,2414,227893.738547
3,1989,111214.428149
4,625,175262.519510
...,...,...
873,1662,177639.222173
874,1234,223838.794251
875,1373,125490.685318
876,1672,120208.846956


In [31]:
test_preds_enet = enet.predict(X_test)
kaggle_submission(test_preds_enet, 'enet_01')

,ID,SalePrice
0,2658,135077.074159
1,2718,182191.972436
2,2414,219457.219187
3,1989,111127.571510
4,625,179287.691137
...,...,...
873,1662,181575.315710
874,1234,243646.707558
875,1373,128393.083099
876,1672,114338.061998


We run through our datasets through 4 regression models; Linear, Ridge, Lasso and ElasticNet.


|Model | Penalty | α | Train Score | Cross-Validation Score | Kaggle Score (Public) |
|---|---|---|---|---|---|
| Linear Regression | - | - | -1.28e+20 | 3.283 e+14| 7.0848e+14 |
| Ridge Regression | L2 | 4.5 | 0.9052 | 24825.2468 | 23432.86209 |
| Lasso Regression | L1 | 65.4625 | 0.9075 | 24682.6912 | 23531.36260 |
| ElasticNet Regression | L1+L2 | 0.5 | 0.8889 | 26986.2282 | 28275.50000 |

For Linear, the scores show that the model is not useful in this dataset, especially when there are 191 features, after pre-processing, and Linear Regression does not regularization and so, the 'noise' that may be present in the model is still taken into consideration.

As for the Ridge and Lasso models, they had the better scores across the scoring metric out of the four models, with Lasso being slightly better. They also scored well on Kaggle upon the first submission on the website. The main reason for this is that these regressions regularize the models and shrink the regression coefficients. Features that are deemed to be unimportant in contributing to the predictive value of the model are given smaller coefficients. Thus, the models tend to have lower variance and can generalise to new data better. Even though both ridge and lasso saw a significant result and prediction score, lasso saw a best score among the three regression models. This is because Lasso shrinks the irrelevant features to zero and allows the model for a better prediction model.

Despite the good results, we will not stop here. There are 191 features in the dataset (58 before OneHot encoding) and with such a high number of features, the model may interpret these features differently. Therefore, to improve the model, we must reduce the complexity of it. Even with regularization, there may still be 'noise' in the model which may affect the model predictability.

We will seek to reduce number of features through recursive feature elimination [(RFE)](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html) which select features by recursively considering smaller and smaller sets of features and filter those with the higher correlation with `'saleprice'`.

Following which, hyperparamter tuning will be done on the reduced features selected by RFE to see if any improvements are seen.

------------------------------------------------
## Model Improvement

### Feature Selection by RFE

In [32]:
# Using similar alpha for ease of computation
model = Lasso(alpha=lasso_cv.alpha_, max_iter=10000)
rfe = RFE(model, n_features_to_select=50)

# Fitting to training data
X_train_rfe = rfe.fit_transform(X_train, y_train)
model.fit(X_train_rfe, y_train)

# Tabulating RFE results
rfe_results = [np.array(X_train.columns), rfe.ranking_]
rfe_results_df = pd.DataFrame(rfe_results).T
rfe_results_df.columns = ['Feature', 'RFE Ranking']

# Finding features used by lasso (1 means feature was used)
rfe_lasso_features = rfe_results_df.loc[rfe_results_df['RFE Ranking'] == 1, 'Feature'].tolist()

#filtering the top features by RFE
X_train_reduced = X_train[rfe_lasso_features]

#Instantiate and fit
lasso_cv_reduced = LassoCV(n_alphas=100, max_iter=10000)
lasso_cv_reduced.fit(X_train_reduced, y_train)

#modelling
lasso_reduced = Lasso(alpha=lasso_cv_reduced.alpha_, max_iter=10000)

print(f'CVS for top 50 is : {np.abs(cross_val_score(lasso_reduced, X_train_reduced, y_train, scoring="neg_root_mean_squared_error").mean())}')
print(f'R2 cross for top 50 is {(cross_val_score(lasso_reduced, X_train_reduced, y_train, cv=3)).mean()}')

lasso_reduced.fit(X_train_reduced, y_train)

print(f'Validation score for top 50 is : {np.sqrt(metrics.mean_squared_error(y_valid, lasso_reduced.predict(X_valid[rfe_lasso_features])))}')
print('-'*30)

CVS for top 50 is : 24027.160799227167
R2 cross for top 50 is 0.9129333449989058
Validation score for top 50 is : 75879.80581327048
------------------------------


In [33]:
lasso_reduced = Lasso(alpha=lasso_cv_reduced.alpha_, max_iter=10000)
lasso_reduced.fit(X_train_reduced, y_train)

Lasso(alpha=62.17652173732708, max_iter=10000)

In [34]:
#predicting for X_test for the top 50 features for lasso
test_preds_lasso_reduced = lasso_reduced.predict(X_test[rfe_lasso_features])
kaggle_submission(test_preds_lasso_reduced, 'lasso_02')

,ID,SalePrice
0,2658,121050.426761
1,2718,165006.617865
2,2414,225534.314439
3,1989,114297.517715
4,625,179929.384843
...,...,...
873,1662,178843.977952
874,1234,225483.814882
875,1373,123746.566750
876,1672,123058.037515


Based on our top 50 features based on RFE, the new Kaggle scores is shown to be a no improvement, despite having an improved scores on our model scoring metrics.
Kaggle score was **23643.55423**

### Hyperparameter Tuning

In [35]:
# Lasso 
lasso_reduced_hyper = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
lasso_params = {'alpha':[0.005, 0.02, 0.03, 0.05, 0.06]}
# define search
search = GridSearchCV(lasso_reduced_hyper , lasso_params, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(X_train[rfe_lasso_features], y_train)

In [36]:
results.best_score_

-16752.102348632536

In [37]:
results.best_params_

{'alpha': 0.06}

In [38]:
# Instantiate
lasso_reduced_hyper = Lasso(alpha=0.06, max_iter=50000)

#check r2 score
print((cross_val_score(lasso_reduced_hyper , X_train[rfe_lasso_features], y_train)).mean())

#cross-validation
np.abs(cross_val_score(lasso_reduced_hyper , X_train[rfe_lasso_features], y_train, scoring='neg_root_mean_squared_error').mean())

0.9147039340409826


23590.34855303373

In [39]:
lasso_reduced_hyper.fit(X_train[rfe_lasso_features], y_train)
test_preds_lasso_hyper = lasso_reduced_hyper.predict(X_test[rfe_lasso_features])

#making kaggle submission dataset
kaggle_submission(test_preds_lasso_hyper , 'lasso_03')

,ID,SalePrice
0,2658,118267.055515
1,2718,164308.939933
2,2414,225176.048107
3,1989,113385.923705
4,625,180408.426183
...,...,...
873,1662,176987.195284
874,1234,222286.354199
875,1373,122764.825593
876,1672,122668.341464


Based on our top 50 features based on RFE and after hyperparameter tuning, the new Kaggle scores is shown to be a no improvement as well, despite having an improved scores on our model scoring metrics.
Kaggle score was **23605.50435**

------------------------------------------------
## Conclusion

The final production model used was the Lasso Regression Model with 50 features selected using Recursive Feature Elimination (Lasso Regression). Even though it showed good metrics scoring result after validating with our valid dataset, the final Kaggle score showed no improvement, especially after the hyperparameter tuning process. This is likely due to the fact that CV models were already used initially to find the best alpha to fit our respective model. Thus, our initial model showed the best result, albeit only a slight change in scores after RFE filtering. 

The final tabulation of the models are as below:

| Model | Penalty | α | Train Score | Cross-Validation Score | Kaggle Score (Public) |
|---|---|---|---|---|---|
| Linear Regression | - | - | -2.0611 | 1.6718e^14 | 7.0848e^14 |
| Ridge Regression | L2 | 4.5 | 0.906 | 24810.7735 | 23429.53810 |
| Lasso Regression | L1 | 65.3995 | 0.9075 | 24670.8261 | 23531.36260 |
| ElasticNet Regression | L1+L2 | 0.5 | 0.8884 | 26979.0956 | 28275.50000 |
| Lasso Regression (after RFE) | L1 | 0.5 | 0.913 | 24019.6993 | 23643.55423 |
| Lasso Regression (after hyperparameter tuning) | L1 | 0.06 | 0.915 | 23582.1916 | 23932.42010 |

From the Kaggle scores, it seems that Ridge was the slightly better model in predicting the saleprice of property on the test dataset, however Lasso seemed to performed better during the modelling stage. Albeit the scores were only off by a few 100 points, both Ridge and Lasso did a respectable job in prediction of the test dataset. 

Our final best Kaggle score was **23531**.

### Interpreting Coefficients

In [40]:
lasso_reduced.coef_

array([ -4949.69114966,   5338.62749157,  -9262.51099374,   7549.51823168,
        -5237.52999279,  -4021.00174735,   6670.34444166,   7521.3863482 ,
         4654.1935624 ,  10248.0210447 ,  13074.80668684,  66266.38700356,
        14924.01891938,  19803.06221956,  31206.79034631,  -2897.81526171,
        10524.78370247,  49027.75360342,  15459.17870324,  -4185.74511626,
         5829.88215562, -29120.13709923,  12898.73625476,  -3507.06920396,
        -7533.75537989,   4127.6801285 ,  -5258.11172707,  -5326.32209941,
        -4997.15145703,   4528.09065289, -11056.69296508,  12452.25589142,
        -5268.0838646 ,   3389.87352296,   3814.61064142,   3498.28230062,
        -7662.16901286,   6261.05048019,   3766.3929208 , -11597.66465045,
        19108.55003905,  11301.1583118 ,  16404.88467823,  11787.61430091,
         8179.08575545,   8290.50975099,   4708.53496601,   6102.42839754,
         6695.1954103 ,   3901.61512126])

In [41]:
lasso_reduced.intercept_

168113.77818385963

In [42]:
#making dataframe with coefficients of the reduced lasso features based on RFE 
coefs = pd.DataFrame([rfe_lasso_features, lasso_reduced.coef_]).T

In [43]:
coefs.columns = ['feature', 'Coefficient']

In [44]:
#top 10 features with positive coefficient
coefs.sort_values(by='Coefficient', ascending=False).head(10)

,feature,Coefficient
11,neighborhood_GrnHill,66266.387004
17,neighborhood_StoneBr,49027.753603
14,neighborhood_NridgHt,31206.790346
13,neighborhood_NoRidge,19803.06222
40,1st_flr_sf_total_bsmt_sf,19108.550039
42,gr_liv_area_2nd_flr_sf,16404.884678
18,bldg_type_1Fam,15459.178703
12,neighborhood_NPkVill,14924.018919
10,neighborhood_Crawfor,13074.806687
22,exterior_1st_BrkFace,12898.736255


In [45]:
#top 10 features with negative coefficient
coefs.sort_values(by='Coefficient', ascending=True).head(10)

,feature,Coefficient
21,roof_style_Mansard,-29120.137099
39,property_age,-11597.66465
30,sale_type_COD,-11056.692965
2,ms_subclass_160,-9262.510994
36,total_bsmt_sf,-7662.169013
24,mas_vnr_type_BrkCmn,-7533.75538
27,garage_type_2Types,-5326.322099
32,sale_type_WD,-5268.083865
26,foundation_CBlock,-5258.111727
4,ms_subclass_85,-5237.529993


Feature coefficients can be simply interpreted as `Bx` in the linear regression equation: `Y = B0 + B1X1 + ... + BxXx + e`
`B0` represents the intercept while `Bx` represents the slope parameter. For example, `'property_age'` impacts on our model as follows: 

`'sale_price'` = -11325.42 + 168148.42(`'overall_qual'`)

Meaning to say, a one unit increase in age of the property before it was sold would relate to a predicted decrease in sale price of $16,8148.42.

From the coefficients, we see that generally neighbourhoods and area/size of property contributes the most in sale price. Generally, locations that are 'prime' will lead to an increase in sale price and same goes to the size of the property as it shows a positive . Conversely, the features that had the greatest negative effect on sale price were the age of the house and certain roof styles (Mansard). The older the age of property, the lower the sale price and the more type of 'less-quality' roof styles (that are least popular among homeowners), the lower the sale price as well. 

Therefore, we can answer our project statement whereby the features listed in the coefficient dataframe has highlighted several similarly-described features has an impact on saleprice of properties in Ames, Iowa; namely **neighbourhood/location, size & area of properties and building types** contributes the most to the housing market.

### Recommendations

Our model is able to predict housing prices relatively well. It can also be fitted based on the needs of the stakeholders. Our dataset does not have some features such as economic indicators, like  employment or wage growth of the area, and interest rates. Based on our outside research, more macro features has an impact on the housing price in the market. An increase in interest rate can influence one's ability to afford a home as the individual's budget will be more focused on managing to pay off additional interest rates (for example, for credit card or short-term loan). [(source)](https://www.opendoor.com/w/blog/factors-that-influence-home-value)

Therefore, we can include such features to better predict housing market based on more macro features, if the needs of our stakeholders require it to. In this case, our stakeholders are catered more to homeowners that may want to find ways to help alleviate their position in selling their properties at a higher price as the features describe more on the description of the property itself. If our stakeholders are more of real estate developers or investors, whose sole purpose is to see intrinsic value due to long-term price appreciation [(source)](https://www.investopedia.com/articles/investing/110614/most-important-factors-investing-real-estate.asp#:~:text=Expected%20cash%20flow%20from%20rental,to%20get%20a%20better%20price), then a more macro view and data is required in which our model is able to run prediction. 

Even with added features, our model will still be able to instantiate them and evaluate if the features are deemed significant in its predictive power. Especially with Lasso Regression Model, it can sieve out noise better by zeroing the coefficient. With proper datasets and training, we can recommend that the model be trained in more macro-related features to boost its predictive power in the housing market.